In [ ]:
import pandas as pd
import numpy

In [ ]:
import spacy

In [ ]:
nlp = spacy.load("en")

In [ ]:
def tokenize(x):
    return nlp(x, parse=False, tag=False, entity=False)


In [ ]:
data_file = 'data/wikitext-2/train.txt'

In [ ]:
def load_file(file_name):
    with open(data_file) as d:
        for line in d:
            yield tokenize(line)
        

In [ ]:
f = load_file(data_file)

In [ ]:
def get_tokens(file):
    tokens = {}
    for sent in file:
        for token in sent:
            try:
                tokens[token.text] += 1
            except:
                tokens[token.text] = 1
    return tokens

In [ ]:
%%time 
tokens = get_tokens(f)

In [ ]:
import math
def norm_probs(tokens):
    s = sum(tokens.values())
    return {k : round(math.log(v/float(s)),4) for k, v in tokens.items()}
def sort_probs(probs):
    return sorted(probs.items(), key=lambda x:x[1], reverse=True)

In [ ]:
probs = norm_probs(tokens)
sorted_probs = sort_probs(probs)

In [ ]:
import nltk

In [ ]:
def get_ngrams(tokens, min_length = 5, max_N=4):
    for token, freq in tokens:
        if len(token) >= min_length:
            splits = []
            for i in range(2, max_N+1):
                splits += [list([''.join(x) for x in nltk.ngrams(list(token), i)])]
            yield token, splits

In [ ]:
def get_pos_encoded(token_len,splits):
    new_splits = {}
    for ng_split in splits:
        for pos, split in enumerate(ng_split):
            if split not in new_splits:
                new_splits[split] = 0
            new_splits[split] += pos/float(token_len)
    return new_splits
        

In [ ]:
%%time
all_splits = {}
for token, splits in get_ngrams(sorted_probs):
    all_splits[token] = get_pos_encoded(len(token), splits)
   

In [ ]:
%%time
result = []
for key, value in all_splits.items():
    row = pd.Series(value).reset_index()
    row.insert(0, 'word', key)
    result.append(row)
    

In [ ]:
result = pd.concat(result, ignore_index=True)

In [ ]:
result

In [ ]:
split_counts = result['index'].value_counts()

In [ ]:
lens = split_counts.index.str.len().unique()
splits_by_len = [split_counts[split_counts.index.str.len() == x] for x in lens]
splits_by_len = [x / (x.sum() * (i+2)) for i,x in enumerate(splits_by_len)]
split_counts = pd.concat(splits_by_len)

In [ ]:
#split_counts = split_counts / split_counts.sum()

In [ ]:
split_quants = split_counts.quantile([0.25, 0.4, 0.6, 0.75, 0.85, 0.95])

In [ ]:
pruned_splits = split_counts[split_counts > split_quants[0.6]]

In [ ]:
pruned_splits.count()

In [ ]:
t = result['index']

In [ ]:
pruned_splits.index.str.len().value_counts()

In [ ]:
t[result['index'].isin(pruned_splits.index)].str.len().value_counts()

In [ ]:
pruned_result = result[result['index'].isin(pruned_splits.index)]

In [ ]:
word_count = len(pruned_result['word'].unique())
s_count = pruned_splits.count()
word_count, s_count

In [ ]:
%%time 
piv = pd.pivot_table(pruned_result, index=['word'], columns=['index']).fillna(0)

In [ ]:
word_split_m = piv.values

In [ ]:
%%time
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=500, n_iter=7, random_state=42)
word_split_red = svd.fit_transform(word_split_m)

In [ ]:
word_split_red.shape

In [ ]:
split_m = numpy.ndarray((word_split_m.shape[1], word_split_red.shape[1]))

In [ ]:
piv.columns[0]

In [ ]:
%%time
a = word_split_m[0]
for i in range(split_m.shape[0]):
    split_v = word_split_red[word_split_m[:,i] > 0].sum(axis=0)
    if numpy.linalg.norm(split_v) > 0:
        split_v = split_v / numpy.linalg.norm(split_v)
    split_m[i] = split_v

In [ ]:
split2index = lambda x : (piv.columns.get_level_values(1) == x).argmax()
index2split = lambda x : piv.columns[x]
word2vector = lambda e : word_split_red[(piv.index == e).argmax()]
split2vector = lambda e : split_m[split2index(e)]
index2word = lambda x : piv.index[x]

In [ ]:
ind = 6652
w = split_m[ind]
splt = index2split(ind)
print(splt)

In [ ]:
w.dot(split_m.T).argsort()[-5:][::-1]

In [ ]:
index2split(13410)

In [ ]:
e = piv.index[3001]

In [ ]:
import scipy
e = 'eating'
evec = word2vector(e)
possible_splits = pruned_result[pruned_result['word'] == e]['index']
distances = sorted(((p, scipy.spatial.distance.cosine(evec, split2vector(p))) for p in possible_splits),key=lambda x:x[1])

In [ ]:
distances

In [ ]:
split2index('Edu')